# Комонады


Комонада - это непустой контейнер, который сфокусирован на определённом элементе

Пример: массив и индекс выделенного элемента 
```scala
 ([1, 2, 3], 1)
```

Так выглядит тайп класс
```scala
class Functor w => Comonad w where
    extract   :: w a -> a
    duplicate :: w a -> w (w a)           -- extend id x
    extend    :: (w a -> b) -> w a -> w b -- fmap f <$> duplicate x

(=>>) :: Comonad w => w a -> (w a -> b) -> w b
```
Очевидно, minimal complete definition: 
 - extract + duplicate
 - extract + extend


**Монады** создают контекст для вычислений

**Комонады** достают что-то из контекста вычислений

добавлю законов от себя
```
f =>= extract   = f // вроде правая единица
extract =>= f   = f // вроде левая единица
(f =>= g) =>= h = f =>= (g =>= h) // вроде ассоциативность
```

# Simple example

```scala
data Identity a = Identity { runIdentity :: a }

instance Comonad Identity where
    extract   = runIdentity
    duplicate = Identity
```

Maybe и \[\] не являются комонадами, потому что данные контейнеры могут быть пустыми!

# Дуализм

```scala
return :: a -> ma // Monad

extract :: a <- m a // Comonad
```
```scala
(>>=) :: m a -> (a -> m b) -> m b // Monad

(=>>) :: m a <- (a <- m b) <- m b // Comonad
```

```scala
join :: m (m a) -> m a // Monad

duplicate :: m (m a) <- m a // Comonad
```

# Zippers

**Zipper** - это контейнер с выделенным элементом и некоторыми правилами для дальнейшего перемещения этого элемента по коллекции

~~~ scala
data [a] = [] | a : [a]
data ListZipper a = LZ [a] a [a]  // allows to focus on a single element

listLeft, listRight :: ListZipper a -> ListZipper a
listLeft  (LZ (a:as) x bs) = LZ as a (x:bs)
listLeft _ = error "listLeft"

listRight (LZ as x (b:bs)) = LZ (x:as) b bs
listRight _ = error "listRight"

listWrite :: a -> ListZipper a -> ListZipper a
listWrite x (LZ ls _ rs) = LZ ls x rs
~~~





Zipper-ы естественным образом являются комонадами

```scala
instance Functor ListZipper where
    fmap f (LZ ls x rs) = LZ (map f ls) (f x) (map f rs)
extract :: ListZipper a -> a
extract (LZ _ x _) = x
```

```scala
iterate :: (a -> a) -> a -> [a] // генерирует массив [a, f a, f (f a), ...] 
iterate = ???

tail :: [a] -> [a] // хвост списка
tail = ???

iterateTail :: (a -> a) -> a -> [a] // [f x, f (f x), ..]
iterateTail f = tail . iterate f

mkZipper :: (v -> v) -> (v -> v) -> v -> ListZipper v
mkZipper genLeft genRight e = LZ (iterateTail genLeft e) e (iterateTail genRight e)

/* 
сохраняет позицию выделенного элемента внешнего зиппера, во внутренних зипперах выделенный элемент меняется

Пример: ListZipper [1] -> ListZipper [ListZipper []     1 [2, 3]]
                   (2)               (ListZipper [1]    2 [3])
                   [3]               [ListZipper [2, 1] 3 []]
*/
duplicate :: ListZipper a -> ListZipper (ListZipper a)
duplicate = mkZipper listLeft listRight
```

# Zipper of zippers



```scala
newtype Grid a = Grid { unGrid :: ListZipper (ListZipper a) }  // 2D grid

up, down :: Grid a -> Grid a
up   (Grid g) = Grid (listLeft  g)
down (Grid g) = Grid (listRight g)  

left, right :: Grid a -> Grid a
left  (Grid g) = Grid (fmap listLeft  g)
right (Grid g) = Grid (fmap listRight g)

gridRead :: Grid a -> a
gridRead (Grid g) = extract $ extract g
 
gridWrite :: a -> Grid a -> Grid a
gridWrite x (Grid g) = Grid $ listWrite newLine g
  where
    oldLine = extract g
    newLine = listWrite x oldLine

horizontal, vertical :: Grid a -> ListZipper (Grid a)
horizontal = mkZipper left right
vertical   = mkZipper up   down

instance Comonad Grid where
    extract :: Grid a -> a
    extract = gridRead

    duplicate :: Grid a -> Grid (Grid a)
    duplicate = Grid . fmap horizontal . vertical
```

# Игра в жизнь

```scala
// True - клетка жива
// False - клетка мертва

aliveCount :: [Bool] -> Int
aliveCount = length . filter id

neighbours :: [Grid a -> Grid a]
neighbours = horizontals ++ verticals ++ liftM2 (.) horizontals verticals
  where horizontals = [left, right]
        verticals   = [up, down]

aliveNeighbours :: Grid Bool -> Int
aliveNeighbours g = aliveCount 
                  $ map (\direction -> extract $ direction g) neighbours

rule :: Grid Bool -> Bool
rule g = case aliveNeighbours g of
     2 -> extract g
     3 -> True
     _ -> False

evolve :: Grid Bool -> Grid Bool
evolve = extend rule
```

# Popular comonads

## Env

```scala
type Pos1D = (Int, Int)  // current position with starting position

start :: Int -> Pos1D
start n = (n, n)

left, right :: Int -> Pos1D -> Int
left  n (_, x) = x - n
right n (_, x) = x + n

data Env e a = Env e a  // just a pair

instance Comonad (Env e) where // aka the Coreader
    extract :: Env e a -> a
    extract (Env _ a) = a

    extend :: (Env e a -> b) -> Env e a -> Env e b
    extend f env@(Env e _) = Env e (f env)

toStart :: Pos1D -> Int
toStart (z, x) = if abs (z - x) >= 10 then z else x

safeRight :: Int -> Pos1D -> Int
safeRight n p = extract $ p =>> right n =>> toStart
```



## Builder pattern

``` scala
type Option = String
data Config = MakeConfig [Option] deriving (Show)
newtype Traced m a = Traced { runTraced :: m -> a }
```

В контейнере лежат объекты типа `a`, а индексированы они указателем типа `m`

``` scala
instance Functor (Traced m) where
  fmap f t = Traced $ f . runTraced t
```

```scala
instance Monoid m => Comonad (Traced m) where
    extract :: Traced m a -> a
    extract (Traced getter) = getter mempty

    extend :: (Traced m a -> b) -> Traced m a -> Traced m b
    extend f (Traced getter) = 
      Traced $ \shift -> f (Traced $ \index -> getter (shift <> index))
```

```scala
type ConfigBuilder = Traced [Option] Config

profile :: ConfigBuilder -> Config
profile builder = runTraced builder ["-prof", "-auto-all"]

goFaster :: ConfigBuilder -> Config
goFaster builder = runTraced builder ["-O2"]
```

# codo-нотация

Её нет, но есть наработки)))
 - sugar
```
method
    wa> expr1
    wb> expr2
    wc> expr3
```
 - desugar
```
   \wa ->
 let wb =      extend (\this -> expr1) wa
     wc =      extend (\this -> expr2) wb
 in extract  $ extend (\this -> expr3) wc
```

 - sugar
```
method
    expr1
    expr2
    expr3
```

 - desugar
```
   \_wa ->
 let _wb =      extend (\this -> expr1) _wa
     _wc =      extend (\this -> expr2) _wb
 in  extract  $ extend (\this -> expr3) _wc
```